In [0]:
import numpy as np
import pandas as pd
import keras
import cv2  

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras import backend as K

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [0]:
! ls gdrive

'My Drive'


In [0]:
sf = pd.read_csv('gdrive/My Drive/Colab Notebooks/fer2013.csv',delimiter=",")
(sf.Usage=="Training").sum()

28709

In [0]:
sf.shape

(35887, 3)

In [0]:
train = []
test = []
y_train = []
y_test = []
for i in range(sf.shape[0]):
  if sf.Usage[i] == "Training":
    train.append(np.array(sf.pixels[i].split(),'float32'))
    y_train.append(sf.emotion[i])
  elif sf.Usage[i] == "PublicTest":
    test.append(np.array(sf.pixels[i].split(),'float32'))
    y_test.append(sf.emotion[i])

In [0]:
train = np.array(train,'float32')
test = np.array(test,'float32')
train.shape

(28709, 2304)

In [0]:
y_train = np.array(y_train,'float32')
y_test = np.array(y_test,'float32')

In [0]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [0]:
train -= np.mean(train)
train /= np.std(train)

test -= np.mean(test)
test /= np.std(test)

In [0]:
num_features = 64
IMAGE_SIZE = 48
no_of_labels = 7

In [0]:
train = train.reshape(-1,48,48,1)

In [0]:
test = test.reshape(-1,48,48,1)

In [0]:
train.shape

(28709, 48, 48, 1)

In [0]:
  def swish_activation(x):
    return (K.sigmoid(x) * x)

In [0]:
def cnn_model():

  model = Sequential()

  model.add(Conv2D(64,(3,3), activation="relu", padding="same", input_shape = (IMAGE_SIZE, IMAGE_SIZE, 1)))
  model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
  model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
  model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
  model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())

  model.add(Dense(64, activation=swish_activation))
  model.add(Dropout(0.4))

  model.add(Dense(no_of_labels , activation='sigmoid'))
  
  model.summary()
  
  return model

In [0]:
my_model = cnn_model()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_73 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 21, 21, 128)    

In [0]:
my_model.compile(loss=categorical_crossentropy,
              optimizer=adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [0]:
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)
tensorboard = TensorBoard(log_dir='./logs')

In [0]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint("weights.h5", monitor='val_acc', verbose=1, save_best_only=True)

In [0]:
my_model.fit(train, y_train,
          batch_size=64,
          epochs=100,
          verbose=1,
          validation_data=(test,y_test),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.9062 - acc: 0.2189 - val_loss: 1.7955 - val_acc: 0.2494

Epoch 00001: val_loss did not improve from 1.06267
Epoch 2/100
28709/28709 [==============================] - 29s 996us/step - loss: 1.7961 - acc: 0.2516 - val_loss: 1.7261 - val_acc: 0.2995

Epoch 00002: val_loss did not improve from 1.06267
Epoch 3/100
28709/28709 [==============================] - 29s 998us/step - loss: 1.6995 - acc: 0.3113 - val_loss: 1.6168 - val_acc: 0.3433

Epoch 00003: val_loss did not improve from 1.06267
Epoch 4/100
28709/28709 [==============================] - 29s 999us/step - loss: 1.5886 - acc: 0.3667 - val_loss: 1.4190 - val_acc: 0.4308

Epoch 00004: val_loss did not improve from 1.06267
Epoch 5/100
28709/28709 [==============================] - 29s 1ms/step - loss: 1.4922 - acc: 0.4125 - val_loss: 1.3927 - val_acc: 0.4447

Epoch 00005: val_loss did not improve from 1.0

In [0]:
from keras.constraints import maxnorm
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center= False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

datagen.fit(train)

In [71]:
augmented_model = cnn_model()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_123 (Conv2D)          (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_124 (Conv2D)          (None, 48, 48, 32)        18464     
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_125 (Conv2D)          (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 12, 12, 96)      

In [0]:
augmented_model.compile(loss='binary_crossentropy',
              optimizer='adam' ,
              metrics=['accuracy'])

In [73]:
augmented_model.fit(train, y_train,
          batch_size=128,
          epochs=100,
          verbose=1,
          validation_data=(test,y_test),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 12s 407us/step - loss: 0.4095 - acc: 0.8518 - val_loss: 0.3835 - val_acc: 0.8571

Epoch 00001: val_acc improved from 0.65227 to 0.85714, saving model to weights.h5
Epoch 2/100
28709/28709 [==============================] - 10s 356us/step - loss: 0.3761 - acc: 0.8596 - val_loss: 0.3271 - val_acc: 0.8731

Epoch 00002: val_acc improved from 0.85714 to 0.87310, saving model to weights.h5
Epoch 3/100
28709/28709 [==============================] - 10s 358us/step - loss: 0.3198 - acc: 0.8770 - val_loss: 0.2910 - val_acc: 0.8841

Epoch 00003: val_acc improved from 0.87310 to 0.88405, saving model to weights.h5
Epoch 4/100
28709/28709 [==============================] - 10s 356us/step - loss: 0.2866 - acc: 0.8861 - val_loss: 0.2793 - val_acc: 0.8879

Epoch 00004: val_acc improved from 0.88405 to 0.88791, saving model to weights.h5
Epoch 5/100
28709/28709 [==============================] - 

In [0]:
fer_json = augmented_model.to_json()
with open("fer.json", "w") as json_file:
  json_file.write(fer_json)

In [0]:
from google.colab import files
files.download('fer.json')   